In diesem Skript wurden Zahlen zur Repräsentation von DE und NL herausgefunden

Generell sind die Zahlen mit vorsicht zu betrachten, da nicht immer alle Gemeinden/Kreise oder Buurten in den Daten waren

In [ ]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import geopandas as gpd
from shapely.geometry import shape

In [ ]:
inkar = pd.read_csv("../data/inkar_2022.csv",sep=";",decimal=",",dtype={'Kennziffer':str,'Kennziffer_EU':str,'Name':str}, low_memory=False)
inkar.head()

Anzahl von Raumbezügen prüfen

In [4]:
raumbezug_selection = ["Gemeindeverbände"]
temp = inkar[inkar['Raumbezug'].isin(raumbezug_selection)]
temp

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
26928,Absolutzahlen,2176,Arbeitslose,Gemeindeverbände,010010000,"Flensburg, Stadt",1998,5784.00
26929,Absolutzahlen,2176,Arbeitslose,Gemeindeverbände,010010000,"Flensburg, Stadt",1999,5155.00
26930,Absolutzahlen,2176,Arbeitslose,Gemeindeverbände,010010000,"Flensburg, Stadt",2000,4831.00
26931,Absolutzahlen,2176,Arbeitslose,Gemeindeverbände,010010000,"Flensburg, Stadt",2001,5049.00
26932,Absolutzahlen,2176,Arbeitslose,Gemeindeverbände,010010000,"Flensburg, Stadt",2002,5078.00
...,...,...,...,...,...,...,...,...
20008619,Wirtschaft,16551,Verweildauer in Beherbergungsbetrieben,Gemeindeverbände,160775052,"Schmölln, Stadt",2016,1.81
20008620,Wirtschaft,16551,Verweildauer in Beherbergungsbetrieben,Gemeindeverbände,160775052,"Schmölln, Stadt",2017,1.94
20008621,Wirtschaft,16551,Verweildauer in Beherbergungsbetrieben,Gemeindeverbände,160775052,"Schmölln, Stadt",2018,1.75
20008622,Wirtschaft,16551,Verweildauer in Beherbergungsbetrieben,Gemeindeverbände,160775052,"Schmölln, Stadt",2019,1.93


In [5]:
anzahl_unique_name = temp['Name'].nunique()
anzahl_zeilen = len(temp)
print(f"Anzahl der Zeilen: {anzahl_zeilen}")
print(f"Anzahl der Raumbezüge: {anzahl_unique_name}")
temp['Name'].unique()


Anzahl der Zeilen: 9351929
Anzahl der Raumbezüge: 4572


array(['Flensburg, Stadt', 'Kiel, Landeshauptstadt', 'Lübeck, Hansestadt',
       ..., 'Gößnitz, Stadt', 'Nobitz', 'Schmölln, Stadt'], dtype=object)

Area von Raumbezügen überprüfen

In [6]:
data = pd.read_csv("out/csvs/csvs/indicator_Bevölkerunggesamt.csv")
data.head(10)

,Kennziffer,Wert,Zeitbezug
0,111,81817498.0,1995
1,111,82012160.0,1996
2,111,82057380.0,1997
3,111,82037011.0,1998
4,111,82163476.0,1999
5,111,82259531.0,2000
6,111,82440309.0,2001
7,111,82536680.0,2002
8,111,82531670.0,2003
9,111,82500850.0,2004


Area für Verwaltungsgebiete

In [7]:
def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_vwg",engine="pyogrio")

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)
municipal_boundaries = municipal_boundaries[['area', 'GEN']]


grouped = municipal_boundaries.groupby(municipal_boundaries['GEN'].str.split(' \(', expand=True)[0])['area'].sum().reset_index()

min_bl = round(grouped["area"].min(),3)
max_bl = round(grouped["area"].max(), 3)
mean_bl = round(grouped["area"].mean(), 3)
median_bl = round(grouped["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
grouped

Min: 0.1, Max: 893.03, Mean: 79.955, Median: 58.66


,0,area
0,Aachen,163.57
1,Aalen,224.03
2,Aar-Einrich,159.35
3,Aarbergen,34.11
4,Abenberg,48.25
...,...,...
4469,Übach-Palenberg,25.90
4470,Überherrn,34.38
4471,Überlingen,99.93
4472,Übersee,30.90


Area für Kreise

In [8]:
def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_krs",engine="pyogrio")

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)
municipal_boundaries = municipal_boundaries[['area', 'GEN']]


grouped = municipal_boundaries.groupby(municipal_boundaries['GEN'].str.split(' \(', expand=True)[0])['area'].sum().reset_index()

min_bl = round(grouped["area"].min(),3)
max_bl = round(grouped["area"].max(), 3)
mean_bl = round(grouped["area"].mean(), 3)
median_bl = round(grouped["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
grouped

Min: 40.41, Max: 5500.53, Mean: 948.851, Median: 816.88


,0,area
0,Ahrweiler,786.81
1,Aichach-Friedberg,779.64
2,Alb-Donau-Kreis,1356.44
3,Altenburger Land,569.83
4,Altenkirchen,640.80
...,...,...
372,Wuppertal,168.37
373,Würzburg,1054.19
374,Zollernalbkreis,916.60
375,Zweibrücken,70.77


Area für Gemeinden

In [9]:
def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_gem",engine="pyogrio")

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)
municipal_boundaries = municipal_boundaries[['area', 'GEN']]


grouped = municipal_boundaries.groupby(municipal_boundaries['GEN'].str.split(' \(', expand=True)[0])['area'].sum().reset_index()

min_bl = round(grouped["area"].min(),3)
max_bl = round(grouped["area"].max(), 3)
mean_bl = round(grouped["area"].mean(), 3)
median_bl = round(grouped["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

Min: 0.1, Max: 893.03, Mean: 34.505, Median: 20.11


Area für Bundesländer

In [10]:
def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_lan",engine="pyogrio")
municipal_boundaries['AGS_int'] = municipal_boundaries.AGS_0.astype(int)

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)
municipal_boundaries = municipal_boundaries[['area', 'GEN']]


grouped = municipal_boundaries.groupby(municipal_boundaries['GEN'].str.split(' \(', expand=True)[0])['area'].sum().reset_index()

#grouped = grouped.iloc[0] + grouped.iloc[1]


min_bl = round(grouped["area"].min(),3)
max_bl = round(grouped["area"].max(), 3)
mean_bl = round(grouped["area"].mean(), 3)
median_bl = round(grouped["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

Min: 419.98, Max: 70591.96, Mean: 23837.114, Median: 20826.28


Area für Regierungsbezirke

In [11]:
def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_rbz",engine="pyogrio")

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)
municipal_boundaries = municipal_boundaries[['area', 'GEN']]


grouped = municipal_boundaries.groupby(municipal_boundaries['GEN'].str.split(' \(', expand=True)[0])['area'].sum().reset_index()

#grouped = grouped.iloc[0] + grouped.iloc[1]


min_bl = round(grouped["area"].min(),3)
max_bl = round(grouped["area"].max(), 3)
mean_bl = round(grouped["area"].mean(), 3)
median_bl = round(grouped["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

Min: 5291.89, Max: 17535.25, Mean: 8498.391, Median: 8008.55


Population für Kreise

In [12]:
#vg250_sta Flächen Ebene Staat
#vg250_lan Flächen Ebene Bundesland
#vg250_rbz Flächen Ebene Regierungsbezirk
#vg250_krs Flächen Ebene Kreis
#vg250_gem Flächen Ebene Gemeinde
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_krs",engine="pyogrio")
municipal_boundaries['AGS_int'] = municipal_boundaries.AGS_0.astype(int)
municipal_boundaries = pd.merge(data, municipal_boundaries,
                               left_on = "Kennziffer", 
                               right_on = "AGS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]

min_bl = municipal_boundaries["Wert"].min()
max_bl = municipal_boundaries["Wert"].max()
mean_bl = municipal_boundaries["Wert"].mean()
median_bl =municipal_boundaries["Wert"].median()

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries

Min: 34001.0, Max: 3664088.0, Mean: 210557.82790697674, Median: 157975.0


,Kennziffer,Wert,Zeitbezug,OBJID,BEGINN,ADE,GF,BSG,ARS,AGS,...,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,WSK,DLM_ID,geometry,AGS_int
50,1001000,89934.0,2020,DEBKGVG200000C75,2022-12-20,4,4,1,01001,01001,...,00,000,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000002R,"MULTIPOLYGON (((526513.753 6075133.412, 526547...",1001000
51,1001000,89934.0,2020,DEBKGVG200000CI9,2022-12-20,4,2,1,01001,01001,...,00,000,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000E5M9,"MULTIPOLYGON (((531016.578 6076768.422, 531473...",1001000
102,1002000,246601.0,2020,DEBKGVG200000C76,2022-12-20,4,4,1,01002,01002,...,00,000,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000E43X,"MULTIPOLYGON (((575841.569 6032148.032, 575869...",1002000
103,1002000,246601.0,2020,DEBKGVG200000CIA,2022-12-20,4,2,1,01002,01002,...,00,000,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000001T,"MULTIPOLYGON (((575848.336 6032304.652, 575965...",1002000
129,1003000,215846.0,2020,DEBKGVG200000C77,2022-12-20,4,4,1,01003,01003,...,00,000,R,DEF03,010030000000,01003000,2006-02-01,DEBKGDL20000DYM9,"MULTIPOLYGON (((623056.151 5983746.445, 623191...",1003000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11075,16073000,102139.0,2020,DEBKGVG200000CI4,2022-12-20,4,4,1,16073,16073,...,00,000,R,DEG0I,160730000000,16073000,2019-01-01,DEBKGDL20000E6O4,"MULTIPOLYGON (((659352.786 5633725.523, 659501...",16073000
11101,16074000,82816.0,2020,DEBKGVG200000CI5,2022-12-20,4,4,1,16074,16074,...,00,000,R,DEG0J,160740000000,16074000,2009-01-01,DEBKGDL20000DZT7,"MULTIPOLYGON (((688961.348 5662575.516, 688976...",16074000
11127,16075000,79632.0,2020,DEBKGVG200000CI6,2022-12-20,4,4,1,16075,16075,...,00,000,R,DEG0K,160750000000,16075000,2009-01-01,DEBKGDL20000E0F6,"MULTIPOLYGON (((699461.667 5630277.321, 699499...",16075000
11153,16076000,96668.0,2020,DEBKGVG200000CI7,2022-12-20,4,4,1,16076,16076,...,00,000,R,DEG0L,160760000000,16076000,2009-01-01,DEBKGDL20000E65V,"MULTIPOLYGON (((724109.327 5650625.772, 724321...",16076000


Population für Gemeinden

In [13]:
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_gem",engine="pyogrio")
municipal_boundaries['AGS_int'] = municipal_boundaries.AGS_0.astype(int)
municipal_boundaries = pd.merge(data, municipal_boundaries,
                               left_on = "Kennziffer", 
                               right_on = "AGS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]
min_bl = municipal_boundaries["Wert"].min()
max_bl = municipal_boundaries["Wert"].max()
mean_bl = municipal_boundaries["Wert"].mean()
median_bl =municipal_boundaries["Wert"].median()
print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries

Min: 34001.0, Max: 3664088.0, Mean: 263660.7192982456, Median: 129557.5


,Kennziffer,Wert,Zeitbezug,OBJID,BEGINN,ADE,GF,BSG,ARS,AGS,...,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,WSK,DLM_ID,geometry,AGS_int
50,1001000,89934.0,2020,DEBKGVG200000008,2022-12-20,6,4,1,010010000000,01001000,...,00,000,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000E5MA,"MULTIPOLYGON (((526513.753 6075133.412, 526547...",1001000
51,1001000,89934.0,2020,DEBKGVG2000008HF,2022-12-20,6,2,1,010010000000,01001000,...,00,000,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000003H,"MULTIPOLYGON (((531016.578 6076768.422, 531473...",1001000
102,1002000,246601.0,2020,DEBKGVG200000009,2022-12-20,6,4,1,010020000000,01002000,...,00,000,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000004J,"MULTIPOLYGON (((575841.569 6032148.032, 575869...",1002000
103,1002000,246601.0,2020,DEBKGVG2000008HG,2022-12-20,6,2,1,010020000000,01002000,...,00,000,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000E6XX,"MULTIPOLYGON (((575848.336 6032304.652, 575965...",1002000
129,1003000,215846.0,2020,DEBKGVG20000000A,2022-12-20,6,4,1,010030000000,01003000,...,00,000,R,DEF03,010030000000,01003000,2006-02-01,DEBKGDL20000DYMA,"MULTIPOLYGON (((623056.151 5983746.445, 623191...",1003000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,16051000,213692.0,2020,DEBKGVG2000007ZW,2022-12-20,6,4,1,160510000000,16051000,...,00,000,R,DEG01,160510000000,16051000,1994-10-12,DEBKGDL20000DXLP,"MULTIPOLYGON (((643368.580 5660697.134, 643363...",16051000
2885,16052000,92126.0,2020,DEBKGVG2000007ZX,2022-12-20,6,4,1,160520000000,16052000,...,00,000,R,DEG02,160520000000,16052000,1994-07-01,DEBKGDL20000DW2U,"MULTIPOLYGON (((717181.863 5651591.598, 717573...",16052000
2911,16053000,110731.0,2020,DEBKGVG2000007ZY,2022-12-20,6,4,1,160530000000,16053000,...,00,000,R,DEG03,160530000000,16053000,1994-07-01,DEBKGDL20000DWHN,"MULTIPOLYGON (((679577.691 5649953.262, 679657...",16053000
2937,16054000,36395.0,2020,DEBKGVG2000007ZZ,2022-12-20,6,4,1,160540000000,16054000,...,00,000,R,DEG04,160540000000,16054000,2019-01-01,DEBKGDL20000E052,"MULTIPOLYGON (((626564.155 5616707.029, 626689...",16054000


Population für Regierungsbezirke

In [14]:
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_rbz",engine="pyogrio")
municipal_boundaries['SDV_ARS_int'] = municipal_boundaries.SDV_ARS.astype('int64')
def konvertiere_kennziffer(kennziffer):
    return int(str(int(kennziffer))[:8])

municipal_boundaries['SDV_ARS_int'] = municipal_boundaries['SDV_ARS_int'].apply(konvertiere_kennziffer)
municipal_boundaries = pd.merge(data, municipal_boundaries,left_on = "Kennziffer", right_on = "SDV_ARS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]

min_bl = municipal_boundaries["Wert"].min()
max_bl = municipal_boundaries["Wert"].max()
mean_bl = municipal_boundaries["Wert"].mean()
median_bl =municipal_boundaries["Wert"].median()

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

municipal_boundaries

Min: 41681.0, Max: 1488202.0, Mean: 318105.45, Median: 180111.0


,Kennziffer,Wert,Zeitbezug,OBJID,BEGINN,ADE,GF,BSG,ARS,AGS,...,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,WSK,DLM_ID,geometry,SDV_ARS_int
25,51110000,620523.0,2020,DEBKGVG200000CJ3,2022-12-20,3,4,1,051,051,...,00,000,R,DEA1,051000000000,05100000,1975-01-01,DEBKGDL20000E3WA,"MULTIPOLYGON (((304587.199 5754146.922, 304780...",51110000
51,53150000,1083498.0,2020,DEBKGVG200000CJ4,2022-12-20,3,4,1,053,053,...,00,000,R,DEA2,053000000000,05300000,1975-01-01,DEBKGDL20000E3WC,"MULTIPOLYGON (((301890.303 5600399.203, 301901...",53150000
77,55150000,316403.0,2020,DEBKGVG200000CJ5,2022-12-20,3,4,1,055,055,...,00,000,R,DEA3,055000000000,05500000,1998-04-01,DEBKGDL20000E44O,"MULTIPOLYGON (((405519.702 5814668.586, 405626...",55150000
103,57660020,74097.0,2020,DEBKGVG200000CJ6,2022-12-20,3,4,1,057,057,...,00,000,R,DEA4,057000000000,05700000,2009-01-01,DEBKGDL20000E65Q,"MULTIPOLYGON (((477607.280 5818768.839, 477707...",57660020
129,59580004,73487.0,2020,DEBKGVG200000CJ7,2022-12-20,3,4,1,059,059,...,00,000,R,DEA5,059000000000,05900000,2009-11-01,DEBKGDL20000E44I,"MULTIPOLYGON (((394543.626 5732696.180, 394595...",59580004
155,64110000,159174.0,2020,DEBKGVG200000CJ8,2022-12-20,3,4,1,064,064,...,00,000,R,DE71,064000000000,06400000,2011-07-01,DEBKGDL20000E3LX,"MULTIPOLYGON (((476929.894 5496705.669, 476984...",64110000
181,65310005,90131.0,2020,DEBKGVG200000CJ9,2022-12-20,3,4,1,065,065,...,00,000,R,DE72,065000000000,06500000,2009-01-01,DEBKGDL20000DZXT,"MULTIPOLYGON (((477948.925 5648954.563, 478206...",65310005
207,66110000,201048.0,2020,DEBKGVG200000CJA,2022-12-20,3,4,1,066,066,...,00,000,R,DE73,066000000000,06600000,2015-01-01,DEBKGDL20000E6X5,"MULTIPOLYGON (((534242.001 5721821.750, 534214...",66110000
233,81110000,630305.0,2020,DEBKGVG200000CJB,2022-12-20,3,4,1,081,081,...,00,000,R,DE11,081000000000,08100000,1997-01-01,DEBKGDL20000E5Z4,"MULTIPOLYGON (((591667.133 5373246.675, 591782...",81110000
259,82120000,308436.0,2020,DEBKGVG200000CJC,2022-12-20,3,4,1,082,082,...,00,000,R,DE12,082000000000,08200000,1983-06-21,DEBKGDL20000E3WH,"MULTIPOLYGON (((477035.141 5496309.300, 476890...",82120000


Population für Bundesländer

In [15]:
auswahl =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
data_bl = data[data["Kennziffer"].isin(auswahl)]
data_bl = data_bl[data_bl["Zeitbezug"] == 2020]

min_bl = data_bl["Wert"].min()
max_bl = data_bl["Wert"].max()
mean_bl = data_bl["Wert"].mean()
median_bl =data_bl["Wert"].median()

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

data_bl

Min: 680130.0, Max: 17925570.0, Mean: 5197189.4375, Median: 3287481.5


,Kennziffer,Wert,Zeitbezug
51,1,2910875.0,2020
77,2,1852478.0,2020
103,3,8003421.0,2020
129,4,680130.0,2020
155,5,17925570.0,2020
181,6,6293154.0,2020
207,7,4098391.0,2020
233,8,11103043.0,2020
259,9,13140183.0,2020
285,10,983991.0,2020


Population für den Bund

In [16]:
data_b = data[data["Kennziffer"] == 111]
data_b = data_b[data_b["Zeitbezug"] == 2020]
data_b

,Kennziffer,Wert,Zeitbezug
25,111,83155031.0,2020


<h1>CBS DATA area and poulation</h1>

In [17]:
import cbsodata

data_netherlands = pd.DataFrame(cbsodata.get_meta('37259ned', 'TypedDataSet'))

In [18]:
data_nl = data_netherlands[['ID', 'RegioS','Perioden','BevolkingOp1Januari_1']]

data_nl = data_nl[data_nl['Perioden'] == '2020JJ00']
data_nl['Perioden'] = data_nl['Perioden'].str.extract(r'(\d{4})')

data_nl = data_nl.replace(0, np.nan, inplace=True)
data_nl = data_nl.dropna()

data_nl['RegioS'] = data_nl['RegioS'].str.strip()

data_nl

,ID,RegioS,Perioden,BevolkingOp1Januari_1
78,78,NL01,2020,17407585.0
160,160,LD01,2020,1729505.0
242,242,LD02,2020,3671379.0
324,324,LD03,2020,8326545.0
406,406,LD04,2020,3680156.0
...,...,...,...,...
318074,318074,GM0879,2020,10690.0
318156,318156,GM0301,2020,24436.0
318484,318484,GM1896,2020,11161.0
318730,318730,GM0642,2020,23043.0


Populationen der Gemeenten

In [19]:
municipal_boundaries_nl = gpd.read_file(filename='GeoJSON:https://service.pdok.nl/cbs/gebiedsindelingen/2021/wfs/v1_0?request=GetFeature&service=WFS&version=1.1.0&outputFormat=json&typeName=gebiedsindelingen:gemeente_gegeneraliseerd', engine="pyogrio")

municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry']]

municipal_boundaries_nl = pd.merge(municipal_boundaries_nl, data_nl,
                               left_on = "statcode", 
                               right_on = "RegioS")

municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry', 'RegioS', 'BevolkingOp1Januari_1', 'Perioden' ]]
municipal_boundaries_nl.columns = ['Code', 'Name', 'geometry', 'Kennziffer', 'Wert', 'Zeitbezug']


min_bl = round(municipal_boundaries_nl["Wert"].min(),3)
max_bl = round(municipal_boundaries_nl["Wert"].max(), 3)
mean_bl = round(municipal_boundaries_nl["Wert"].mean(), 3)
median_bl = round(municipal_boundaries_nl["Wert"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries_nl


Min: 461.0, Max: 872757.0, Mean: 32948.448, Median: 20720.0


,Code,Name,geometry,Kennziffer,Wert,Zeitbezug
0,GM0014,Groningen,"MULTIPOLYGON (((245194.691 592594.007, 245344....",GM0014,232874.0,2020
1,GM0014,Groningen,"MULTIPOLYGON (((245194.691 592594.007, 245344....",GM0014,115838.0,2020
2,GM0014,Groningen,"MULTIPOLYGON (((245194.691 592594.007, 245344....",GM0014,117036.0,2020
3,GM0034,Almere,"MULTIPOLYGON (((146891.056 493291.709, 147422....",GM0034,211893.0,2020
4,GM0034,Almere,"MULTIPOLYGON (((146891.056 493291.709, 147422....",GM0034,105130.0,2020
...,...,...,...,...,...,...
1048,GM1970,Noardeast-Fryslân,"MULTIPOLYGON (((207686.537 595425.623, 207640....",GM1970,22768.0,2020
1049,GM1970,Noardeast-Fryslân,"MULTIPOLYGON (((207686.537 595425.623, 207640....",GM1970,22460.0,2020
1050,GM1978,Molenlanden,"MULTIPOLYGON (((123712.581 439998.364, 124188....",GM1978,43909.0,2020
1051,GM1978,Molenlanden,"MULTIPOLYGON (((123712.581 439998.364, 124188....",GM1978,22256.0,2020


Population der Provincies

In [20]:
municipal_boundaries_nl = gpd.read_file(filename='GeoJSON:https://service.pdok.nl/cbs/gebiedsindelingen/2021/wfs/v1_0?request=GetFeature&service=WFS&version=1.1.0&outputFormat=json&typeName=gebiedsindelingen:provincie_gegeneraliseerd', engine="pyogrio")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry']]

municipal_boundaries_nl = pd.merge(data_nl, municipal_boundaries_nl,
                               left_on = "RegioS", 
                               right_on = "statcode")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry', 'RegioS', 'BevolkingOp1Januari_1', 'Perioden' ]]
municipal_boundaries_nl.columns = ['Code', 'Name', 'geometry', 'Kennziffer', 'Wert', 'Zeitbezug']


min_bl = round(municipal_boundaries_nl["Wert"].min(),3)
max_bl = round(municipal_boundaries_nl["Wert"].max(), 3)
mean_bl = round(municipal_boundaries_nl["Wert"].mean(), 3)
median_bl = round(municipal_boundaries_nl["Wert"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries_nl.drop_duplicates()

Min: 190435.0, Max: 3708696.0, Mean: 967088.056, Median: 617911.5

,Code,Name,geometry,Kennziffer,Wert,Zeitbezug
0,PV20,Groningen,"MULTIPOLYGON (((276560.583 585160.682, 276458....",PV20,585866.0,2020
1,PV20,Groningen,"MULTIPOLYGON (((276560.583 585160.682, 276458....",PV20,292444.0,2020
2,PV20,Groningen,"MULTIPOLYGON (((276560.583 585160.682, 276458....",PV20,293422.0,2020
3,PV21,Fryslân,"MULTIPOLYGON (((154047.947 565045.368, 154282....",PV21,649957.0,2020
4,PV21,Fryslân,"MULTIPOLYGON (((154047.947 565045.368, 154282....",PV21,325951.0,2020
5,PV21,Fryslân,"MULTIPOLYGON (((154047.947 565045.368, 154282....",PV21,324006.0,2020
6,PV22,Drenthe,"MULTIPOLYGON (((228930.078 579661.963, 229126....",PV22,493682.0,2020
7,PV22,Drenthe,"MULTIPOLYGON (((228930.078 579661.963, 229126....",PV22,245475.0,2020
8,PV22,Drenthe,"MULTIPOLYGON (((228930.078 579661.963, 229126....",PV22,248207.0,2020
9,PV23,Overijssel,"MULTIPOLYGON (((182721.334 511713.845, 182821....",PV23,1162406.0,2020


Population der COROPS

In [21]:
municipal_boundaries_nl = gpd.read_file(filename='GeoJSON:https://service.pdok.nl/cbs/gebiedsindelingen/2021/wfs/v1_0?request=GetFeature&service=WFS&version=1.1.0&outputFormat=json&typeName=gebiedsindelingen:coropgebied_gegeneraliseerd', engine="pyogrio")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry']]

municipal_boundaries_nl = pd.merge(data_nl, municipal_boundaries_nl,
                               left_on = "RegioS", 
                               right_on = "statcode")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry', 'RegioS', 'BevolkingOp1Januari_1', 'Perioden' ]]
municipal_boundaries_nl.columns = ['Code', 'Name', 'geometry', 'Kennziffer', 'Wert', 'Zeitbezug']


min_bl = round(municipal_boundaries_nl["Wert"].min(),3)
max_bl = round(municipal_boundaries_nl["Wert"].max(), 3)
mean_bl = round(municipal_boundaries_nl["Wert"].mean(), 3)
median_bl = round(municipal_boundaries_nl["Wert"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries_nl.drop_duplicates()

Min: 22927.0, Max: 1454720.0, Mean: 290126.417, Median: 200596.5


,Code,Name,geometry,Kennziffer,Wert,Zeitbezug
0,CR01,Oost-Groningen,"MULTIPOLYGON (((276560.583 585160.682, 276458....",CR01,135208.0,2020
1,CR01,Oost-Groningen,"MULTIPOLYGON (((276560.583 585160.682, 276458....",CR01,67320.0,2020
2,CR01,Oost-Groningen,"MULTIPOLYGON (((276560.583 585160.682, 276458....",CR01,67888.0,2020
3,CR02,Delfzijl en omgeving,"MULTIPOLYGON (((253004.946 603887.823, 253797....",CR02,45857.0,2020
4,CR02,Delfzijl en omgeving,"MULTIPOLYGON (((253004.946 603887.823, 253797....",CR02,22930.0,2020
...,...,...,...,...,...,...
115,CR39,Zuid-Limburg,"MULTIPOLYGON (((185595.689 341998.739, 185819....",CR39,294150.0,2020
116,CR39,Zuid-Limburg,"MULTIPOLYGON (((185595.689 341998.739, 185819....",CR39,303034.0,2020
117,CR40,Flevoland,"MULTIPOLYGON (((164912.900 510024.315, 164641....",CR40,423021.0,2020
118,CR40,Flevoland,"MULTIPOLYGON (((164912.900 510024.315, 164641....",CR40,211643.0,2020


Population der Landsdeelen

In [22]:
municipal_boundaries_nl = gpd.read_file(filename='GeoJSON:https://service.pdok.nl/cbs/gebiedsindelingen/2021/wfs/v1_0?request=GetFeature&service=WFS&version=1.1.0&outputFormat=json&typeName=gebiedsindelingen:landsdeel_gegeneraliseerd', engine="pyogrio")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry']]

municipal_boundaries_nl = pd.merge(data_nl, municipal_boundaries_nl,
                               left_on = "RegioS", 
                               right_on = "statcode")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry', 'RegioS', 'BevolkingOp1Januari_1', 'Perioden' ]]
municipal_boundaries_nl.columns = ['Code', 'Name', 'geometry', 'Kennziffer', 'Wert', 'Zeitbezug']


min_bl = round(municipal_boundaries_nl["Wert"].min(),3)
max_bl = round(municipal_boundaries_nl["Wert"].max(), 3)
mean_bl = round(municipal_boundaries_nl["Wert"].mean(), 3)
median_bl = round(municipal_boundaries_nl["Wert"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries_nl.drop_duplicates()

Min: 863870.0, Max: 8326545.0, Mean: 2901264.167, Median: 1840332.0


,Code,Name,geometry,Kennziffer,Wert,Zeitbezug
0,LD01,Noord-Nederland,"MULTIPOLYGON (((154047.947 565045.368, 154282....",LD01,1729505.0,2020
1,LD01,Noord-Nederland,"MULTIPOLYGON (((154047.947 565045.368, 154282....",LD01,863870.0,2020
2,LD01,Noord-Nederland,"MULTIPOLYGON (((154047.947 565045.368, 154282....",LD01,865635.0,2020
3,LD02,Oost-Nederland,"MULTIPOLYGON (((169898.402 486171.144, 169615....",LD02,3671379.0,2020
4,LD02,Oost-Nederland,"MULTIPOLYGON (((169898.402 486171.144, 169615....",LD02,1830858.0,2020
5,LD02,Oost-Nederland,"MULTIPOLYGON (((169898.402 486171.144, 169615....",LD02,1840521.0,2020
6,LD03,West-Nederland,"MULTIPOLYGON (((24837.490 381472.461, 26874.37...",LD03,8326545.0,2020
7,LD03,West-Nederland,"MULTIPOLYGON (((24837.490 381472.461, 26874.37...",LD03,4113290.0,2020
8,LD03,West-Nederland,"MULTIPOLYGON (((24837.490 381472.461, 26874.37...",LD03,4213255.0,2020
9,LD04,Zuid-Nederland,"MULTIPOLYGON (((119172.354 380543.324, 119234....",LD04,3680156.0,2020


Area der Landsdeelen

In [23]:
municipal_boundaries_nl = gpd.read_file(filename='GeoJSON:https://service.pdok.nl/cbs/gebiedsindelingen/2021/wfs/v1_0?request=GetFeature&service=WFS&version=1.1.0&outputFormat=json&typeName=gebiedsindelingen:landsdeel_gegeneraliseerd', engine="pyogrio")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry']]

municipal_boundaries_nl = pd.merge(data_nl, municipal_boundaries_nl,
                               left_on = "RegioS", 
                               right_on = "statcode")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry', 'RegioS', 'Perioden' ]]
municipal_boundaries_nl.columns = ['Code', 'Name', 'geometry', 'Kennziffer', 'Zeitbezug']

def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries_nl['area'] = municipal_boundaries_nl['geometry'].apply(calculate_area)

min_bl = round(municipal_boundaries_nl["area"].min(),3)
max_bl = round(municipal_boundaries_nl["area"].max(), 3)
mean_bl = round(municipal_boundaries_nl["area"].mean(), 3)
median_bl = round(municipal_boundaries_nl["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries_nl

Min: 7263.47, Max: 9993.06, Mean: 8771.16, Median: 8914.055


,Code,Name,geometry,Kennziffer,Zeitbezug,area
0,LD01,Noord-Nederland,"MULTIPOLYGON (((154047.947 565045.368, 154282....",LD01,2020,8604.76
1,LD01,Noord-Nederland,"MULTIPOLYGON (((154047.947 565045.368, 154282....",LD01,2020,8604.76
2,LD01,Noord-Nederland,"MULTIPOLYGON (((154047.947 565045.368, 154282....",LD01,2020,8604.76
3,LD02,Oost-Nederland,"MULTIPOLYGON (((169898.402 486171.144, 169615....",LD02,2020,9993.06
4,LD02,Oost-Nederland,"MULTIPOLYGON (((169898.402 486171.144, 169615....",LD02,2020,9993.06
5,LD02,Oost-Nederland,"MULTIPOLYGON (((169898.402 486171.144, 169615....",LD02,2020,9993.06
6,LD03,West-Nederland,"MULTIPOLYGON (((24837.490 381472.461, 26874.37...",LD03,2020,9223.35
7,LD03,West-Nederland,"MULTIPOLYGON (((24837.490 381472.461, 26874.37...",LD03,2020,9223.35
8,LD03,West-Nederland,"MULTIPOLYGON (((24837.490 381472.461, 26874.37...",LD03,2020,9223.35
9,LD04,Zuid-Nederland,"MULTIPOLYGON (((119172.354 380543.324, 119234....",LD04,2020,7263.47


Area der Provincies

In [24]:
municipal_boundaries_nl = gpd.read_file(filename='GeoJSON:https://service.pdok.nl/cbs/gebiedsindelingen/2021/wfs/v1_0?request=GetFeature&service=WFS&version=1.1.0&outputFormat=json&typeName=gebiedsindelingen:provincie_gegeneraliseerd', engine="pyogrio")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry']]

municipal_boundaries_nl = pd.merge(data_nl, municipal_boundaries_nl,
                               left_on = "RegioS", 
                               right_on = "statcode")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry', 'RegioS', 'Perioden' ]]
municipal_boundaries_nl.columns = ['Code', 'Name', 'geometry', 'Kennziffer', 'Zeitbezug']

def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries_nl['area'] = municipal_boundaries_nl['geometry'].apply(calculate_area)

min_bl = round(municipal_boundaries_nl["area"].min(),3)
max_bl = round(municipal_boundaries_nl["area"].max(), 3)
mean_bl = round(municipal_boundaries_nl["area"].mean(), 3)
median_bl = round(municipal_boundaries_nl["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries_nl.drop_duplicates()

Min: 1466.35, Max: 5118.18, Mean: 2923.719, Median: 2772.18


,Code,Name,geometry,Kennziffer,Zeitbezug,area
0,PV20,Groningen,"MULTIPOLYGON (((276560.583 585160.682, 276458....",PV20,2020,2393.23
3,PV21,Fryslân,"MULTIPOLYGON (((154047.947 565045.368, 154282....",PV21,2020,3531.19
6,PV22,Drenthe,"MULTIPOLYGON (((228930.078 579661.963, 229126....",PV22,2020,2680.34
9,PV23,Overijssel,"MULTIPOLYGON (((182721.334 511713.845, 182821....",PV23,2020,3408.53
12,PV24,Flevoland,"MULTIPOLYGON (((164912.900 510024.315, 164641....",PV24,2020,1466.35
15,PV25,Gelderland,"MULTIPOLYGON (((169898.402 486171.144, 169615....",PV25,2020,5118.18
18,PV26,Utrecht,"MULTIPOLYGON (((130059.000 479450.000, 130070....",PV26,2020,1553.71
21,PV27,Noord-Holland,"MULTIPOLYGON (((150846.937 478090.885, 150899....",PV27,2020,2864.02
24,PV28,Zuid-Holland,"MULTIPOLYGON (((97460.999 412337.999, 96725.94...",PV28,2020,2967.13
27,PV29,Zeeland,"MULTIPOLYGON (((24837.490 381472.461, 26874.37...",PV29,2020,1838.49


Area der COROP's

In [25]:
municipal_boundaries_nl = gpd.read_file(filename='GeoJSON:https://service.pdok.nl/cbs/gebiedsindelingen/2021/wfs/v1_0?request=GetFeature&service=WFS&version=1.1.0&outputFormat=json&typeName=gebiedsindelingen:coropgebied_gegeneraliseerd', engine="pyogrio")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry']]

municipal_boundaries_nl = pd.merge(data_nl, municipal_boundaries_nl,
                               left_on = "RegioS", 
                               right_on = "statcode")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry', 'RegioS', 'Perioden' ]]
municipal_boundaries_nl.columns = ['Code', 'Name', 'geometry', 'Kennziffer', 'Zeitbezug']

def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries_nl['area'] = municipal_boundaries_nl['geometry'].apply(calculate_area)

min_bl = round(municipal_boundaries_nl["area"].min(),3)
max_bl = round(municipal_boundaries_nl["area"].max(), 3)
mean_bl = round(municipal_boundaries_nl["area"].mean(), 3)
median_bl = round(municipal_boundaries_nl["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries_nl.drop_duplicates()


Min: 116.08, Max: 1842.14, Mean: 877.116, Median: 888.91


,Code,Name,geometry,Kennziffer,Zeitbezug,area
0,CR01,Oost-Groningen,"MULTIPOLYGON (((276560.583 585160.682, 276458....",CR01,2020,770.06
3,CR02,Delfzijl en omgeving,"MULTIPOLYGON (((253004.946 603887.823, 253797....",CR02,2020,273.16
6,CR03,Overig Groningen,"MULTIPOLYGON (((209948.244 597852.448, 209920....",CR03,2020,1350.01
9,CR04,Noord-Friesland,"MULTIPOLYGON (((146531.724 584861.591, 145929....",CR04,2020,1531.71
12,CR05,Zuidwest-Friesland,"MULTIPOLYGON (((154047.947 565045.368, 154282....",CR05,2020,992.81
15,CR06,Zuidoost-Friesland,"MULTIPOLYGON (((207789.061 574296.102, 207768....",CR06,2020,1006.68
18,CR07,Noord-Drenthe,"MULTIPOLYGON (((228930.078 579661.963, 229126....",CR07,2020,1061.28
21,CR08,Zuidoost-Drenthe,"MULTIPOLYGON (((259847.326 556204.621, 260417....",CR08,2020,923.65
24,CR09,Zuidwest-Drenthe,"MULTIPOLYGON (((216585.467 549045.953, 218649....",CR09,2020,695.41
27,CR10,Noord-Overijssel,"MULTIPOLYGON (((182721.334 511713.845, 182821....",CR10,2020,1479.81


Area der Gemeenten

In [26]:
municipal_boundaries_nl = gpd.read_file(filename='GeoJSON:https://service.pdok.nl/cbs/gebiedsindelingen/2021/wfs/v1_0?request=GetFeature&service=WFS&version=1.1.0&outputFormat=json&typeName=gebiedsindelingen:gemeente_gegeneraliseerd', engine="pyogrio")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry']]

municipal_boundaries_nl = pd.merge(data_nl, municipal_boundaries_nl,
                               left_on = "RegioS", 
                               right_on = "statcode")
municipal_boundaries_nl = municipal_boundaries_nl[['statcode', 'statnaam', 'geometry', 'RegioS', 'Perioden' ]]
municipal_boundaries_nl.columns = ['Code', 'Name', 'geometry', 'Kennziffer', 'Zeitbezug']

def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries_nl['area'] = municipal_boundaries_nl['geometry'].apply(calculate_area)

min_bl = round(municipal_boundaries_nl["area"].min(),3)
max_bl = round(municipal_boundaries_nl["area"].max(), 3)
mean_bl = round(municipal_boundaries_nl["area"].mean(), 3)
median_bl = round(municipal_boundaries_nl["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries_nl.drop_duplicates()

Min: 7.79, Max: 575.17, Mean: 99.178, Median: 73.38


,Code,Name,geometry,Kennziffer,Zeitbezug,area
0,GM1680,Aa en Hunze,"MULTIPOLYGON (((248973.683 567523.258, 250623....",GM1680,2020,278.78
3,GM0358,Aalsmeer,"MULTIPOLYGON (((113740.090 474295.305, 113599....",GM0358,2020,32.32
6,GM0197,Aalten,"MULTIPOLYGON (((235537.451 443533.464, 236251....",GM0197,2020,97.07
9,GM0059,Achtkarspelen,"MULTIPOLYGON (((210504.119 586245.320, 210593....",GM0059,2020,104.01
12,GM0482,Alblasserdam,"MULTIPOLYGON (((102822.359 432749.868, 103345....",GM0482,2020,10.05
...,...,...,...,...,...,...
1038,GM0879,Zundert,"MULTIPOLYGON (((108880.837 394230.253, 109058....",GM0879,2020,121.23
1041,GM0301,Zutphen,"MULTIPOLYGON (((210285.174 464470.806, 210524....",GM0301,2020,42.90
1044,GM1896,Zwartewaterland,"MULTIPOLYGON (((204591.939 519987.922, 203774....",GM1896,2020,87.10
1047,GM0642,Zwijndrecht,"MULTIPOLYGON (((102988.618 426751.564, 103300....",GM0642,2020,22.76
